In [339]:
import copy
import json
import math
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
%matplotlib inline

In [344]:
with open('input18', 'r') as f:
    lines = [l.strip() for l in f.readlines()]

## Part 1

In [192]:
class Tree(object):
    def __init__(self):
        self.left = None
        self.right = None
    def __str__(self):
        return f'[{self.left},{self.right}]'

In [125]:
def to_tree(number):
    tree = Tree()
    if type(number[0]) is int:
        tree.left = number[0]
    else:
        tree.left = to_tree(number[0])
    if type(number[1]) is int:
        tree.right = number[1]
    else:
        tree.right = to_tree(number[1])
    return tree

In [126]:
def replace_node(tree, search, replace):
    to_visit = [tree]
    while len(to_visit) > 0:
        node = to_visit.pop(0)
        if type(node) is Tree:
            if node.left == search:
                node.left = replace
                return tree
            elif node.right == search:
                node.right = replace
                return tree
            else:
                to_visit.append(node.left)
                to_visit.append(node.right)
    return tree

In [127]:
def add_to_prev_leaf(tree, base_node, addition, direction):
    to_visit = [(tree, None, None)]
    last_leaf = (None, None)
    while len(to_visit) > 0:
        node, parent, child = to_visit.pop(0)
        if type(node) is Tree:
            if node == base_node:
                break
            else:
                if direction == 0:
                    to_visit.insert(0, (node.right, node, 1))
                    to_visit.insert(0, (node.left, node, 0))
                else:
                    to_visit.insert(0, (node.left, node, 0))
                    to_visit.insert(0, (node.right, node, 1))
        else:
            last_leaf = (parent, child)
    if last_leaf[0] is not None:
        if last_leaf[1] == 0:
            last_leaf[0].left += addition
        else:
            last_leaf[0].right += addition
    return tree

In [217]:
def explode(tree):
    to_visit = [(tree, 0, None, None)]
    while len(to_visit) > 0:
        node, depth, parent, direction = to_visit.pop(0)
        if type(node) is Tree:
            if depth == 4:
                if parent is not None:
                    tree = add_to_prev_leaf(tree, node, node.left, 0)
                    tree = add_to_prev_leaf(tree, node, node.right, 1)
                tree = replace_node(tree, node, 0)
                return tree, True
                break
            else:
                to_visit.insert(0, (node.right, depth + 1, node, 0))
                to_visit.insert(0, (node.left, depth + 1, node, 1))
    return tree, False

In [299]:
def split(tree):
    to_visit = [(tree, None, None)]
    while len(to_visit) > 0:
        node, parent, child = to_visit.pop(0)
        if type(node) is int:
            if node >= 10:
                new_node = Tree()
                new_node.left = int(math.floor(node / 2))
                new_node.right = int(math.ceil(node / 2))
                if child == 0:
                    parent.left = new_node
                else:
                    parent.right = new_node
                return tree, Tree
        else:
            to_visit.insert(0, (node.right, node, 1))
            to_visit.insert(0, (node.left, node, 0))
    return tree, False

In [304]:
def reduce(tree):
    while True:
        tree, did_explode = explode(tree)
        if did_explode:
            continue
        tree, did_split = split(tree)
        if did_split:
            continue
        break
    return tree

In [132]:
def add(n1, n2):
    tree = Tree()
    tree.left = n1
    tree.right = n2
    result = reduce(tree)
    return result

In [241]:
def magnitude(t):
    if type(t) is int:
        return t
    else:
        return 3 * magnitude(t.left) + 2 * magnitude(t.right)

In [286]:
def print_tree(tree, depth=0):
    if type(tree) is int:
        print('| ' * depth + '|_' + str(tree))
    else:
        print('| ' * depth + '|_.')
        print_tree(tree.left, depth + 1)
        print_tree(tree.right, depth + 1)

In [316]:
trees = []
for line in lines:
    trees.append(to_tree(eval(line)))

result = add(trees.pop(0), trees.pop(0))
while len(trees) > 0:
    result = add(result, trees.pop(0))
print(magnitude(result))

4116


## Part 2

In [345]:
trees = []
for line in lines:
    trees.append(to_tree(eval(line)))

In [346]:
magnitudes = []
for i in range(len(trees)):
    for j in range(len(trees)):
        if i == j:
            continue
        magnitudes.append(magnitude(add(copy.deepcopy(trees[i]), copy.deepcopy(trees[j]))))
print(max(magnitudes))

4638
